In [ ]:
# enable plotting in notebook
%matplotlib notebook

# Check ensemble of water system using python arrays

## Prepare checks

Start by importing the `physical_validation` package.
Please refer to http://physical-validation.readthedocs.io/ for the full documentation. We will also need `numpy`.

In [7]:
import physical_validation as pv
import numpy as np

Our test system consists of 900 H2O molecules whose bonds are fully constrained. During the simulation, we kept the translation of the center of mass to zero, so we need to reduce the number of degrees of freedom by 3.

In [2]:
system = pv.data.SystemData(
    natoms=900*3,
    nconstraints=900*3,
    ndof_reduction_tra=3,
    ndof_reduction_rot=0
)

The physical validation tests need some information on the units that were used in the simulation. While the strings are only used for output, the conversion respective to GROMACS units are relevant for the calculations. Please see the documentation for more information.

In [3]:
units = pv.data.UnitData(
    kb=8.314462435405199e-3,
    energy_str='kJ/mol',
    energy_conversion=1.0,
    length_str='nm',
    length_conversion=1.0,
    volume_str='nm^3',
    volume_conversion=1.0,
    temperature_str='K',
    temperature_conversion=1.0,
    pressure_str='bar',
    pressure_conversion=1.0,
    time_str='ps',
    time_conversion=1.0
)

For the python array example, we will only look at the simulations performed under NVT conditions using the velocity-rescale thermostat. There are two simulations: One ran at 300K, and one ran at 308K.

In [4]:
ensemble_1 = pv.data.EnsembleData(
    ensemble='NVT',
    natoms=900*3,
    volume=3.01125**3,
    temperature=300
)
ensemble_2 = pv.data.EnsembleData(
    ensemble='NVT',
    natoms=900*3,
    volume=3.01125**3,
    temperature=308
)
dir_1 = 'md_NVT_vr_1'
dir_2 = 'md_NVT_vr_2'

In this example, we will assume that we have the kinetic energy, the potential energy, and the total energy as 1-dimensional numpy arrays. These might have been obtained, e.g., from the python API of a simulation code, or from other python-based analysis tools. Here, for the sake of having an easy example, we will simply read them from files.

In [8]:
kin_ene_1 = []
pot_ene_1 = []
tot_ene_1 = []
with open(dir_1 + '/kinetic_energy.dat') as f:
    for line in f:
        kin_ene_1.append(float(line.strip()))
with open(dir_1 + '/potential_energy.dat') as f:
    for line in f:
        pot_ene_1.append(float(line.strip()))
with open(dir_1 + '/total_energy.dat') as f:
    for line in f:
        tot_ene_1.append(float(line.strip()))
kin_ene_1 = np.array(kin_ene_1)
pot_ene_1 = np.array(pot_ene_1)
tot_ene_1 = np.array(tot_ene_1)

kin_ene_2 = []
pot_ene_2 = []
tot_ene_2 = []
with open(dir_2 + '/kinetic_energy.dat') as f:
    for line in f:
        kin_ene_2.append(float(line.strip()))
with open(dir_2 + '/potential_energy.dat') as f:
    for line in f:
        pot_ene_2.append(float(line.strip()))
with open(dir_2 + '/total_energy.dat') as f:
    for line in f:
        tot_ene_2.append(float(line.strip()))
kin_ene_2 = np.array(kin_ene_2)
pot_ene_2 = np.array(pot_ene_2)
tot_ene_2 = np.array(tot_ene_2)

Using these arrays, we create the observables data structures:

In [9]:
observables_1 = pv.data.ObservableData(
    kinetic_energy = kin_ene_1,
    potential_energy = pot_ene_1,
    total_energy = tot_ene_1
)
observables_2 = pv.data.ObservableData(
    kinetic_energy = kin_ene_2,
    potential_energy = pot_ene_2,
    total_energy = tot_ene_2
)

We can now create a representation of the simulation results by creating the object explicitly, i.e. without the help of a parser:

In [10]:
result_1 = pv.data.SimulationData(
    units=units, ensemble=ensemble_1,
    system=system, observables=observables_1
)
result_2 = pv.data.SimulationData(
    units=units, ensemble=ensemble_2,
    system=system, observables=observables_2
)

## Run checks

As with the other parsers, we can now test, for example, the kinetic energy using the created simulation result data structure, where

* the first input is the simulation results read in,
* `strict` determines whether we test the full distribution (`True`) or only determine the mean and the variance of the distribution (`False`),
* `verbosity` sets the level of detail of the output  (with `verbosity=0` being quiet and `verbosity=3` being the most chatty),
* `filename` is being used to plot the resulting distribution for visual inspection.

In [11]:
print('==> Kinetic energy test of simulation ' + dir_1)
pv.kinetic_energy.distribution(result_1, strict=False, verbosity=2,
                               filename='ke_flat_vr_NVT_1')
print('==> Kinetic energy test of simulation ' + dir_2)
pv.kinetic_energy.distribution(result_2, strict=False, verbosity=2,
                               filename='ke_flat_vr_NVT_2')

==> Kinetic energy test of simulation md_NVT_vr_1
After equilibration, decorrelation and tail pruning, 97.27% (17509 frames) of original Kinetic energy remain.
Kinetic energy distribution check (non-strict)
Analytical distribution (T=300.00 K):
 * mu: 6730.97 kJ/mol
 * sigma: 129.57 kJ/mol
Trajectory:
 * mu: 6729.71 +- 0.96 kJ/mol
   T(mu) = 299.94 +- 0.04 K
 * sigma: 130.01 +- 0.73 kJ/mol
   T(sigma) = 301.01 +- 1.69 K
==> Kinetic energy test of simulation md_NVT_vr_2
After equilibration, decorrelation and tail pruning, 90.29% (16253 frames) of original Kinetic energy remain.
Kinetic energy distribution check (non-strict)
Analytical distribution (T=308.00 K):
 * mu: 6910.47 kJ/mol
 * sigma: 133.03 kJ/mol
Trajectory:
 * mu: 6911.29 +- 1.02 kJ/mol
   T(mu) = 308.04 +- 0.05 K
 * sigma: 133.35 +- 0.84 kJ/mol
   T(sigma) = 308.74 +- 1.95 K


(0.8079285996049095, 0.37868329464899947)

We can also test the distribution of the potential energy. While the first two inputs to the tests are the parsed simulation results, `verbosity` sets the level of detail of the output  (with `verbosity=0` being quiet and `verbosity=3` being the most chatty), and `filename` is being used to plot the resulting distribution for visual inspection.

In [12]:
print('==> Potential energy test')
pv.ensemble.check(result_1, result_2,
                  verbosity=2, filename='pe_flat_vr_NVT')

==> Potential energy test
Analytical slope of ln(P_2(U)/P_1(U)): 0.01041319


/home/pascal/physval/pyenv/3.6-jupyter/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


After equilibration, decorrelation and tail pruning, 82.43% (14839 frames) of original Trajectory 1 remain.
After equilibration, decorrelation and tail pruning, 82.77% (14899 frames) of original Trajectory 2 remain.
Overlap is 89.9% of trajectory 1 and 88.0% of trajectory 2.
A rule of thumb states that a good overlap is found when dT/T = (2*kB*T)/(sig),
where sig is the standard deviation of the energy distribution.
For the current trajectories, dT = 8.0, sig1 = 175.3 and sig2 = 178.1.
According to the rule of thumb, given T1, a good dT is dT = 8.5, and
                                given T2, a good dT is dT = 8.9.
Rule of thumb estimates that dT = 8.7 would be optimal (currently, dT = 8.0)
Linear Fit Analysis (analytical error)
Free energy
    373.52454 +/- 4.55809
Estimated slope                  |  True slope
    0.010420  +/- 0.000127       |  0.010413 
    (0.05 quantiles from true slope)
Estimated dT                     |  True dT
    8.0    +/- 0.1               |  8.0   
Maxi

[0.6607683512027682]